In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [23]:
train_dir = "C:/Users/Ronit Das/Desktop/NSUT work/Video Analytics/Vandalism detection/Video Dataset/Test data"
test_dir = "C:/Users/Ronit Das/Desktop/NSUT work/Video Analytics/Vandalism detection/Video Dataset/Train data"


In [32]:
# Modify the data generators to use the new preprocessing function
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

# Define a function to preprocess videos into sequences of frames
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []

    # Read and process every NUM_FRAMES frames
    for _ in range(NUM_FRAMES):
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_WIDTH, IMG_HEIGHT))
        frames.append(frame)

        # Skip FRAME_SKIP frames
        for _ in range(FRAME_SKIP - 1):
            cap.read()

    cap.release()
    return np.array(frames)

# Define functions to generate video sequences for training and testing
def generate_video_sequences(generator):
    while True:
        for x_batch, y_batch in generator:
            video_sequences = []
            for video_paths in x_batch:
                video_frames = [preprocess_video(video_path) for video_path in video_paths]
                video_sequences.append(video_frames)
            video_sequences = np.array(video_sequences)
            yield video_sequences, y_batch

# Train the model using video sequences
history = model.fit(
    generate_video_sequences(train_generator),
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=generate_video_sequences(test_generator),
    validation_steps=len(test_generator))

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(
    generate_video_sequences(test_generator),
    steps=len(test_generator))
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Found 0 images belonging to 5 classes.


Found 0 images belonging to 5 classes.


ValueError: You must call `compile()` before using the model.

In [ ]:
# Set up parameters for video processing
NUM_FRAMES = 20  # Number of frames per video
FRAME_SKIP = 5   # Skip every 5 frames to reduce redundancy

# Define a function to preprocess videos into sequences of frames
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []

    # Read and process every NUM_FRAMES frames
    for _ in range(NUM_FRAMES):
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_WIDTH, IMG_HEIGHT))
        frames.append(frame)

        # Skip FRAME_SKIP frames
        for _ in range(FRAME_SKIP - 1):
            cap.read()

    cap.release()
    return np.array(frames)

# Define functions to generate video sequences for training and testing
def generate_video_sequences(generator):
    while True:
        for x_batch, y_batch in generator:
            video_sequences = []
            for video_path in x_batch:
                video_frames = preprocess_video(video_path)
                video_sequences.append(video_frames)
            yield np.array(video_sequences), y_batch

# Modify the data generators to use the new preprocessing function
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

# Train the model using video sequences
history = model.fit(
    generate_video_sequences(train_generator),
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=generate_video_sequences(test_generator),
    validation_steps=len(test_generator))

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(
    generate_video_sequences(test_generator),
    steps=len(test_generator))
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE)

In [ ]:
# Save the trained model
model.save("video_classifier_model")

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)